In [7]:
# STEP 0: Setup for Google Colab
!pip install ipywidgets --quiet

# ✅ NEW: Clone your GitHub repo and navigate into it
import os, sys, shutil

# Remove old copy if it exists
if os.path.exists("RealOrAI-DogDetector"):
    shutil.rmtree("RealOrAI-DogDetector")

!git clone https://github.com/zhaoq23/RealOrAI-DogDetector.git

# Change directory to repo and add to Python path
os.chdir("RealOrAI-DogDetector")
sys.path.append(".")

# ✅ Now your model import will work
import torch
from IPython.display import display
import ipywidgets as widgets
from models import BayesianMLP

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


Cloning into 'RealOrAI-DogDetector'...
remote: Enumerating objects: 136, done.
remote: Counting objects: 100% (91/91), done.
remote: Compressing objects: 100% (89/89), done.
remote: Total 136 (delta 35), reused 10 (delta 2), pack-reused 45 (from 1)
Receiving objects: 100% (136/136), 105.85 MiB | 26.31 MiB/s, done.
Resolving deltas: 100% (39/39), done.


In [8]:
!pip install ipywidgets --quiet

import torch
from torch.utils.data import DataLoader
from IPython.display import display
import ipywidgets as widgets
from models import BayesianMLP

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [9]:
def load_model(model_path="plusdiff_model.pt"):
    model = BayesianMLP()
    model.load_state_dict(torch.load(model_path, map_location=device))
    model.eval()
    return model


In [10]:
def classify_feature_tensor(tensor, model):
    tensor = tensor.to(device)
    with torch.no_grad():
        output = model(tensor)
        prob = torch.sigmoid(output).item()
        prediction = "AI-GENERATED 🤖" if prob >= 0.5 else "REAL 🐶"
        print(f"Prediction: {prediction}  |  Probability: {prob:.2f}")


In [11]:
upload = widgets.FileUpload(accept='.pt', multiple=False)

def on_upload_change(change):
    if upload.value:
        file_info = next(iter(upload.value.values()))
        with open("temp.pt", "wb") as f:
            f.write(file_info['content'])

        # Load the .pt file and extract features
        data = torch.load("temp.pt")
        if isinstance(data, tuple):  # (features, label)
            tensor = data[0]
        else:
            tensor = data

        if len(tensor.shape) == 1:
            tensor = tensor.unsqueeze(0)  # Add batch dimension if needed

        model = load_model()
        classify_feature_tensor(tensor, model)

upload.observe(on_upload_change, names='value')
display(upload)


FileUpload(value={}, accept='.pt', description='Upload')

RuntimeError: mat1 and mat2 shapes cannot be multiplied (2658x384 and 385x128)